https://cssanalytics.wordpress.com/2020/12/23/how-should-trend-followers-adjust-to-the-modern-environment-enter-adaptive-momentum/

In [ ]:
from datetime import datetime
from itertools import product

import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

from numba import njit


def sharpe_ratio(ret, N=252, verbose=True):
    ret = ret[ret!=0]
    sr = np.mean(ret) / np.std(ret) * np.sqrt(N)
    if verbose:
        print(f'SR (rf=0): {sr}')
        print(f'VOL: {np.std(ret) * np.sqrt(N)}')
    return sr

def compound_annual_growth_rate(cpnl, verbose=True):
    t = ((cpnl.index[-1] - cpnl.index[0]) / 365.25)
    t = (t.seconds / (24 * 60 * 60) + t.days)
    FV = cpnl.values[-1] / cpnl.values[0]
    cagr = FV ** (1/t) - 1
    if verbose:
        print(f'CAGR: {cagr}')
    return cagr


def dd_curve(df):
    df['dd'] = dd_curve_numba(df.cpnl.values, df.ret.values)
    return df

@njit
def dd_curve_numba(cpnl, ret):
    N = len(cpnl)
    i=1
    hw = cpnl[1]
    dd = np.zeros(N)
    while i < N:
        if hw > cpnl[i]:
            # dd[i] = dd[i-1] * (1 + ret[i])
            dd[i] = cpnl[i] / hw - 1
        else:
            dd[i] = 0
            hw = cpnl[i]
        i+=1
    return dd
    

@njit
def ecdf_dd_curve_numba(cpnl, ret):
    N = len(cpnl)
    i=1
    hw = cpnl[1]
    dd = np.zeros(N)
    ecdf = np.zeros(N)
    while i < N:
        if hw > cpnl[i]:
            # dd[i] = dd[i-1] * (1 + ret[i])
            dd[i] = cpnl[i] / hw - 1
        else:
            dd[i] = 0
            hw = cpnl[i]

        ecdf[i] = np.sum(dd[i] < dd[:i]) / i

        i+=1
    return dd, ecdf


def performance_stats(df, N=12, verbose = True):
    sr = sharpe_ratio(df.ret, N, verbose)
    cagr = compound_annual_growth_rate(df.cpnl, verbose)
    df = dd_curve(df)
    max_dd = np.abs(df.dd.min())
    if verbose:
        print(f'Max draw down {max_dd}')
    pm = pd.DataFrame({'sr': sr, 'cagr': cagr, 'max_dd': max_dd}, index=[0])
    return df, pm



def aema(df, asset, f_win = 50, p_win=10):

    ecdf=df.ecdf ** 2
    sma_slow = df[asset].ewm(span=200).mean()
    sma_fast = df[asset].ewm(span=f_win).mean()
    df['ama'] = (sma_slow * (1 - ecdf)) + (sma_fast * ecdf)

    sma_slow = df[asset].ewm(span=f_win).mean()
    sma_fast = df[asset].ewm(span=p_win).mean()
    df['pama'] = (sma_slow * (1 - ecdf)) + (sma_fast * ecdf)

    return df.pama <= df.ama

def ama_performance(df, mask, verbose=False, leverage = False):
    df['ret'] = df.asset_ret.shift(-1).fillna(0)
    df.loc[mask, 'ret'] = 0

    leverage = 0.1 / ((df.asset_ret.pow(2)).rolling(20).mean().pow(1/2) * np.sqrt(252))
    leverage[leverage > 1] = 1
    df['ret'] = df.ret * leverage.fillna(1)

    df['cpnl'] = (1 + df.ret).cumprod()
    df, pm = performance_stats(df, N=252, verbose=verbose)
    return df, pm



def optimise_ama(prices_df, asset):
    pms = []

    for f, p in product(np.arange(0, 55, 5)[1:], np.arange(0, 55, 5)[1:]):
        try:
            asset_ret_df = prices_df[[asset]].dropna().copy()
            asset_ret_df = asset_ret_df.loc[asset_ret_df.index.year > 2010]
            asset_ret_df[asset] /= asset_ret_df[asset].values[0]
            asset_ret_df['asset_ret'] = asset_ret_df[asset].pct_change().fillna(0)

            _ = ecdf_dd_curve_numba(asset_ret_df[asset].values, asset_ret_df.asset_ret.values)
            asset_ret_df['dd'] = _[0]
            asset_ret_df['ecdf'] = _[1]
            mask = aema(asset_ret_df, asset, f, p)
            df, pm = ama_performance(asset_ret_df, mask, verbose=False)
            pm['price'] = p
            pm['fast'] = f
            pms.append(pm)
        except Exception as e:
            print(repr(e))
            # print('failed')
    pms = pd.concat(pms).reset_index(drop=True)

    # print(pms.describe())
    print(asset)
    # print(pms.iloc[pms.sr.idxmax()])
    print(pms.iloc[pms.cagr.idxmax()])
    # print(pms.iloc[pms.max_dd.idxmin()])

    # best_pm = pms.iloc[pms.sr.idxmax()]
    best_pm = pms.iloc[pms.cagr.idxmax()]
    # best_pm = pms.iloc[pms.max_dd.idxmin()]

    asset_ret_df = prices_df[[asset]].dropna().copy()
    asset_ret_df[asset] /= asset_ret_df[asset].values[0]
    asset_ret_df = asset_ret_df.loc[asset_ret_df.index.year > 2010]
    asset_ret_df['asset_ret'] = asset_ret_df[asset].pct_change().fillna(0)

    _ = ecdf_dd_curve_numba(asset_ret_df[asset].values, asset_ret_df.asset_ret.values)
    asset_ret_df['dd'] = _[0]
    asset_ret_df['ecdf'] = _[1]
    mask = aema(asset_ret_df, asset, best_pm.fast, best_pm.price)
    asset_ret_df, pm = ama_performance(asset_ret_df, mask, verbose=True)

    return asset_ret_df

us_syms = ['TLT', 'IEF', 'DBC', 'VNQ', 'VEA', 'VWO', 'RPAR', 'DBMF']
au_syms = ['SPY.AX', 'NDQ.AX', 'MOAT.AX', 'VTS.AX', 'GOLD.AX'] #, 'CBA.AX', 'ANZ.AX'] #, 'BOND.AX', 'GOLD.AX']
assets = us_syms + au_syms
full_prices_df = yf.download(tickers=assets,
                        end=datetime.now())

prices_df = full_prices_df['Adj Close'].copy()

usd_aud_df = pd.read_csv('../trading_data/AUD_USD Historical Data.csv').set_index('Date')
usd_aud_df.index = pd.to_datetime(usd_aud_df.index, dayfirst=True)

sym_usd_df = prices_df[us_syms].copy()
sym_usd_df = pd.merge(sym_usd_df, left_index = True, right = usd_aud_df[['Price']], right_index=True, how='left')
sym_usd_df[us_syms] = sym_usd_df[us_syms].div(sym_usd_df.Price, axis=0)

prices_df = pd.concat([prices_df[au_syms], sym_usd_df[us_syms]], axis=1)

asset_ret_dfs = []

for asset in assets:
    asset_ret_df = optimise_ama(prices_df, asset)
    asset_ret_dfs.append(asset_ret_df)

print('#### Stratergy ####')
strat_ret_df = pd.concat(asset_ret_dfs)[['ret']].dropna().groupby('Date')[['ret']].mean()
strat_ret_df['cpnl'] = (1 + strat_ret_df.ret).cumprod()

strat_ret_df, pm = performance_stats(strat_ret_df, N=252)

_ = ecdf_dd_curve_numba(strat_ret_df.cpnl.values, asset_ret_df.ret.values)

strat_ret_df['dd'] = _[0]
strat_ret_df['ecdf'] = _[1]

plot_df = strat_ret_df.melt(ignore_index=False, value_vars=[ 'cpnl', 'dd', 'ecdf'])
plot_df['facet'] = ''
# plot_df.loc[plot_df.variable.isin([asset, 'ama', 'pama']), 'facet'] = 'price'
plot_df.loc[plot_df.variable.isin(['cpnl']), 'facet'] = 'cpnl'
plot_df.loc[plot_df.variable.isin(['dd', 'ecdf']), 'facet'] = 'signal'
# plot_df.loc[plot_df.variable== 'dd', 'value'] *= 10

fig = px.line(plot_df, y='value', color='variable', facet_row='facet', log_y=True)#.show()
fig['layout']['yaxis']['type'] = None
fig.update_yaxes(matches=None)
fig.update_layout(height = 1000)
fig.show()

# assets = ['SPY.AX', 'NDQ.AX', 'MOAT.AX', 'VTS.AX']
# full
# SR (rf=0): 1.286139137866483
# VOL: 0.12406134127454162
# CAGR: 0.15226279969113943
# Max draw down 0.17633005604325958

# 0.3
# SR (rf=0): 1.2784816214127621
# VOL: 0.1231225867515726
# CAGR: 0.150078154327824
# Max draw down 0.16970333017715267

# 0.2
# SR (rf=0): 1.2770194642565702
# VOL: 0.12009998729372003
# CAGR: 0.1461438920686451
# Max draw down 0.15041759767576401

# 0.1
# SR (rf=0): 1.294449796349582
# VOL: 0.09081798811745248
# CAGR: 0.1116825483700532
# Max draw down 0.09329467336968778